In [ ]:
from dataclasses import dataclass
from typing import Callable
import re
import operator


@dataclass
class Monkey:
    id: int
    items: list[int]
    operation: Callable
    divisbleBy: int
    iftrue: int
    iffalse: int
    inspects: int = 0


def eval_op(e1: str, op: str, e2: str):
    f = operator.add if op == "+" else operator.mul
    return lambda item: f(item if e1 == "old" else int(e1), item if e2 == "old" else int(e2))


PATTERN = re.compile(
    "Monkey (?P<monkey>\d+):\s+Starting items: (?P<items>\d+(, \d+)*)\s+Operation: new = (?P<e1>[a-z0-9]+) (?P<op>[\+\-\*]) (?P<e2>[a-z0-9]+)\s+Test: divisible by (?P<divisible>\d+)\s+If true: throw to monkey (?P<iftrue>\d+)\s+If false: throw to monkey (?P<iffalse>\d+)", re.MULTILINE)


def parse_monkey(raw: str) -> Monkey:
    m = re.match(PATTERN, raw)
    return Monkey(
        int(m.group("monkey")),
        [int(x.strip()) for x in m.group("items").split(", ")],
        eval_op(m.group("e1"), m.group("op"), m.group("e2")),
        int(m.group("divisible")),
        int(m.group("iftrue")),
        int(m.group("iffalse"))
    )


def parse():
    buf = ""
    for line in open("input.txt"):
        if line == "\n":
            yield parse_monkey(buf)
            buf = ""
        else:
            buf += line
    yield parse_monkey(buf)


In [ ]:
monkies = list(parse())
for round in range(20):
    for monkey in monkies:
        for item in monkey.items:
            new_item = monkey.operation(item) // 3
            monkies[monkey.iftrue if new_item % monkey.divisbleBy == 0 else monkey.iffalse].items.append(new_item)
            monkey.inspects += 1
        monkey.items = []

m1, m2 = sorted(monkies, key=lambda x: x.inspects, reverse=True)[:2]
m1.inspects * m2.inspects

In [ ]:
monkies = list(parse())

import functools
modmod = functools.reduce(operator.mul, [monkey.divisbleBy for monkey in monkies])

for round in range(10000):
    for monkey in monkies:
        for item in monkey.items:
            new_item = monkey.operation(item) % modmod
            monkies[monkey.iftrue if new_item % monkey.divisbleBy == 0 else monkey.iffalse].items.append(new_item)
            monkey.inspects += 1
        monkey.items = []

m1, m2 = sorted(monkies, key=lambda x: x.inspects, reverse=True)[:2]
m1.inspects * m2.inspects